<a href="https://colab.research.google.com/github/YousraAshour/Head-Pose-Detection/blob/main/Head_Pose_Estimation_Supervised_Machine_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q opencv-python
!pip install -q mediapipe
!pip install -q mediapipe

In [ ]:
import os
import cv2
import glob
import warnings
import mediapipe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import scipy.io as sio
from pathlib import Path
from math import cos, sin

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
# link of the data
# http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
from zipfile import ZipFile
names_path=[]
with ZipFile('/content/drive/MyDrive/ITI/Projects/Head-Pose-Estimation/AFLW2000-3D.zip', 'r') as zipObject:
   
   # Extract all the contents of zip file in different directory
   zipObject.extractall('/content/')
   listOfFileNames = zipObject.namelist()

   for fileName in listOfFileNames:
       if fileName.endswith('.jpg'):
           # Extract a single file from zip
           names_path.append(zipObject.extract(fileName).split('.')[0])
len(names_path)           
          

2000

In [70]:
dir_path = '/content/AFLW2000/'
paths = os.listdir(dir_path)
names = [x.split('.')[0] for x in paths if x.endswith('.jpg')]

# names= [Path(f).stem for f in glob.glob("/content/AFLW2000/*.mat")]

In [71]:
faceModule = mediapipe.solutions.face_mesh
len(names_path)

2000

In [90]:
landmarks=[]
res_imgs=[]
X=[]
Y=[]
poses=[]
for path in names_path:
    with faceModule.FaceMesh(static_image_mode=True) as face:
      image = cv2.imread(path + '.jpg')
      points = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      
      if points.multi_face_landmarks: 

          landmark=points.multi_face_landmarks[0]
          res_imgs.append(path)
          x_local=[]
          y_local=[]
          for points in landmark.landmark:
            x= points.x
            y= points.y

            shape = image.shape 
            relative_x = int(x * shape[1])
            relative_y = int(y * shape[0])
            # X_features
            x_local.append(relative_x)
            # Y_features
            y_local.append(relative_y)

          
          X.append(np.array(x_local))
          Y.append(np.array(y_local))

          mat_file = sio.loadmat(path +'.mat')
          pose_para = mat_file["Pose_Para"][0][:3]
          poses.append(np.array(pose_para)) 
          

In [94]:
poses=np.array(poses)
X=np.array(X)
Y=np.array(Y)

pitch_label = poses[0:,0]

yaw_label = poses[0:,1]

roll_label = poses[0:,2]
print(X.shape)
print(Y.shape)
print(poses.shape)

(1853, 468)
(1853, 468)
(1853, 3)


In [95]:
Mean_X = X - X[:,1].reshape(-1,1)
Mean_Y = Y - Y[:,1].reshape(-1,1)

X_0 = X[:,0]
X_100 = X[:,100]
Y_0 = Y[:,0]
Y_100 = Y[:,100]

distance = np.linalg.norm(np.array((X_0,Y_0)) - np.array((X_100,Y_100)),axis = 0).reshape(-1,1)
Norm_X = Mean_X / distance
Norm_Y = Mean_Y / distance

print(Norm_X.shape)
print(Norm_Y.shape)

features = np.hstack([Norm_X,Norm_Y])
print(features.shape)

(1853, 468)
(1853, 468)
(1853, 936)


In [96]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img